# **Arrhythmia Analysis**
Arrhythmia merupakan penyakit detak jantung yang tidak tepat, apakah tidak teratur, terlalu cepat atau terlalu lambat. Aritmia jantung terjadi ketika impuls listrik di jantung tidak bekerja dengan baik. Sehingga, penyakit ini begitu berbahaya karena dapat menjadikan ancaman untuk nyawa penderitanya.

Sehingga, diperlukannya akurasi untuk mendeteksi keberadaan aritmia dan klasifikasi aritmia sangat penting. Perbedaan value parameter dapat di ekstrak dari waveform ECG dan dapat digunakan bersamaan dengan informasi pasiennya seperti umur, riwayat penyakit, dsb untuk mendeteksi adanya aritmia. Namun, terkadang sulit untuk dokter mengamati panjangnya durasi record dari ECG tersebut dan menemukan menit-menit irregularitas atau ketidakteraturan detak jantung. Sehingga, diperlukannya penggunaan machine learning untuk automasi diagnosis penyakit aritmia akan membantu para dokter.

**Bagian 01 - Data yang akan dipilih**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import recall_score
from sklearn.model_selection import StratifiedKFold
kFold = StratifiedKFold(n_splits=5)
from sklearn import metrics

In [2]:
data = pd.read_csv('arrhythmia.csv',header=None)
data.head()

FileNotFoundError: ignored